# Compare RNN map & Transformer LFMCS to true observations

**Purpose:** We want to compare metrics by the RNN maps to predictions by the transformer model.

**Date:** July 29, 2024


In [5]:
import pandas as pd
from hydroDL import kPath

In [14]:
sites_path = f'{kPath.dirVeg}model/data/singleDaily-nadgrid/info.csv'
sites_df = pd.read_csv(sites_path)

true_obs_dir = f'{kPath.dirVeg}inference/true_obs_interpolate_semimonthly/' 
rnn_maps_dir = f'{kPath.dirVeg}inference/rnn_maps/'


In [12]:
# TESTING: choose an arbitrary site
site_id = sites_df.iloc[0].siteId

In [28]:
site_obs_path = f'{true_obs_dir}{site_id}.csv'
site_rnn_path = f'{rnn_maps_dir}{site_id}.csv'

site_obs_df = pd.read_csv(site_obs_path, index_col=0)
site_rnn_df = pd.read_csv(site_rnn_path, index_col=0)

In [51]:
compare_df = pd.merge(site_obs_df, site_rnn_df, left_index=True, right_index=True)
compare_df['true_lfmc'] = compare_df['lfmc_x']
compare_df['rnn_lfmc'] = compare_df['lfmc_y']
compare_df = compare_df.drop(['lfmc_x', 'lfmc_y'], axis=1)
compare_df = compare_df.dropna()
compare_df = compare_df[compare_df.rnn_lfmc != 0]

In [52]:

compare_df


,true_lfmc,rnn_lfmc
2019-07-01,99.000000,94
2019-08-01,106.000000,96
2019-08-15,106.666667,96
2019-09-01,107.333333,95
2019-09-15,108.000000,93
2019-10-15,97.214286,93
2019-11-01,97.428571,95
2019-11-15,97.642857,93
2019-12-01,97.857143,92
2019-12-15,98.071429,91
